# Preparation

In [6]:
%run Preparation.ipynb
df152 = pd.read_csv("../data/2017-09-05.1.52.csv")
df152 = df152.rename(index=str, columns={'playerId': 'sessionId'})

# Player filtering

In [7]:
players = df152.loc[:, playerFilteringColumns]
players = players.dropna(how='any')
players.head()

,playerId,customData.localplayerguid,customData.platform,serverTime


In [8]:
players.shape[0]

0

In [9]:
players = players[~players['customData.localplayerguid'].isin(excludedIDs)];
players.shape[0]

0

## Sessions (filtered)

In [10]:
sessionscount = players["sessionId"].nunique()
sessionscount

KeyError: 'sessionId'

## Sessions of dev IDs

## Unique players

In [ ]:
uniqueplayers = players['customData.localplayerguid']
uniqueplayers = uniqueplayers.unique()
uniqueplayers.shape[0]

In [ ]:
#uniqueplayers

## Unique platforms

In [ ]:
uniqueplatforms = players['customData.platform'].unique()
uniqueplatforms

## Checkpoints passed / furthest checkpoint (unfiltered)

In [ ]:
checkpoints = df152.loc[:, checkpointsRelevantColumns]
checkpoints = checkpoints[checkpoints['type']=='reach'].loc[:,['section','sessionId']]
checkpoints = checkpoints[checkpoints['section'].str.startswith('tutorial', na=False)]
checkpoints = checkpoints.groupby("sessionId")
checkpoints = checkpoints.max()
checkpoints.head()

In [ ]:
maxCheckpointTable = pd.DataFrame({"maxCheckpoint" : checkpoints.values.flatten()})
maxCheckpointCounts = maxCheckpointTable["maxCheckpoint"].value_counts()
maxCheckpointCounts['Start'] = None
maxCheckpointCounts = maxCheckpointCounts.sort_index()
print '\nmaxCheckpointCounts=\n{0}'.format(maxCheckpointCounts)

In [ ]:
%run plotUtilities.ipynb

In [ ]:
maxCheckpointCountsTable = pd.DataFrame({"maxCheckpoint" : maxCheckpointCounts.values})
maxCheckpointCountsTableCount = maxCheckpointCountsTable.sum(0)[0]
maxCheckpointCountsTableCount

In [ ]:
checkpoints.count()

In [ ]:
maxCheckpointCountsTable.head()

In [ ]:
maxCheckpointCountsTable.describe()

In [ ]:
genericTreatment( maxCheckpointCountsTable, "best checkpoint reached", "game sessions", 0, maxCheckpointCountsTableCount, False, True )

## Session starts

In [ ]:
#starts = df152.loc[:, checkpointsRelevantColumns]
#starts = checkpoints[checkpoints['type']=='start'].loc[:,['playerId']]
#starts = checkpoints[checkpoints['section'].str.startswith('tutorial', na=False)]
#starts = checkpoints.groupby("playerId")
#starts = checkpoints.max()
#starts.head()

In [ ]:
startTutorial1Count = sessionscount
neverReachedGameSessionCount = startTutorial1Count - maxCheckpointCountsTableCount
fullMaxCheckpointCounts = maxCheckpointCounts
fullMaxCheckpointCounts['Start'] = neverReachedGameSessionCount
fullMaxCheckpointCountsTable = pd.DataFrame({"fullMaxCheckpoint" : fullMaxCheckpointCounts.values})

genericTreatment( fullMaxCheckpointCountsTable, "best checkpoint reached", "game sessions", 0, startTutorial1Count, False, True )

print '\nfullMaxCheckpointCountsTable=\n{0}'.format(fullMaxCheckpointCountsTable)
fullMaxCheckpointCountsTable.describe()

## Duration

Duration of playing sessions

In [ ]:
durations = players.groupby("sessionId").agg({ "serverTime": [ np.min, np.max  ] })
durations["duration"] = pd.to_datetime(durations["serverTime"]["amax"]) - pd.to_datetime(durations["serverTime"]["amin"])
durations["duration"] = durations["duration"].map(lambda x: np.timedelta64(x, 's'))
durations = durations.sort_values(by=['duration'], ascending=[False])
durations.head()

Duration plot

In [ ]:
durations.loc[:,'duration']
durations = durations[4:]
durations["duration_seconds"] = durations["duration"].map(lambda x: pd.Timedelta(x).seconds)
maxDuration = np.max(durations["duration_seconds"])
durations["duration_rank"] = durations["duration_seconds"].rank(ascending=False)
durations.plot(x="duration_rank", y="duration_seconds")
plt.xlabel("game session")
plt.ylabel("time played (s)")
plt.legend('')
plt.xlim(0, sessionscount)
plt.ylim(0, maxDuration)
durations["duration_seconds"].describe()
durations.head()